# 1、数据清理

In [1]:
import pandas as pd
import numpy as np

#path='D:/code/test_2018.10/machine learning/datasets/Data.csv'
path='C:/Users/Taylor/新建文件夹/machine learning/datasets/Data.csv'
data=pd.read_csv(path,engine='python')
x=data.iloc[:,:-1].values
y=data.iloc[:,3].values

#空值处理
from sklearn.preprocessing import Imputer
im=Imputer(missing_values='NaN',strategy='mean',axis=0)
x[:,1:3]=im.fit_transform(x[:,1:3])

#字符编码
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
xenc=OneHotEncoder(categorical_features=[0],sparse = False)
yenc=LabelEncoder()
x[:,0]=yenc.fit_transform(x[:,0])
x=xenc.fit_transform(x)
y=yenc.fit_transform(y)

#归一化
from sklearn.preprocessing import StandardScaler  #Z变换
from sklearn.preprocessing import MinMaxScaler    #0-1
from sklearn.preprocessing import Normalizer      #正则化
std=StandardScaler()
minmax=MinMaxScaler()
norm = Normalizer()

# 2、数据集划分

In [2]:
#留出法 holdout
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [3]:
#K折交叉验证
from sklearn.model_selection import KFold
k=5
kf=KFold(n_splits=k)
#kf.get_n_splits(x)
for train_index,test_index in kf.split(x):
    x_train,x_test=x[train_index],x[test_index]
    y_train,y_test=y[train_index],y[test_index]

In [4]:
#分层K折交叉验证
from sklearn.model_selection import StratifiedKFold
k=5
skf=StratifiedKFold(n_splits=k)
for train_index,test_index in skf.split(x,y):
    x_train,x_test=x[train_index],x[test_index]
    y_train,y_test=y[train_index],y[test_index]
    print(y_train)

[0 0 1 1 0 1 0 1]
[0 1 0 1 0 1 0 1]
[0 1 0 1 0 1 0 1]
[0 1 0 0 1 1 0 1]
[0 1 0 0 1 1 0 1]


# 3、模型评估

In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.datasets import load_iris
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB

iris = load_iris()

xx=iris.data
yy=iris.target

xx=xx[yy<2]
yy=yy[yy<2]

k=5
lr=LogisticRegression()
tree=DecisionTreeRegressor()
by = GaussianNB() 
svc=SVC(kernel='linear')

#cv为整数默认调用分层k折，可以自己定义cv=KFold(n_splits=k)
scoring='accuracy'   #accuracy,f1,precision,recall,roc_auc
lr_score=cross_val_score(lr,xx,yy,cv=k,scoring=scoring)
tree_score=cross_val_score(tree,xx,yy,cv=k,scoring=scoring)
by_score=cross_val_score(by,xx,yy,cv=k,scoring=scoring)
svc_score=cross_val_score(svc,xx,yy,cv=k,scoring=scoring)

print(lr_score.mean())
print(tree_score.mean())
print(by_score.mean())
print(svc_score.mean())



1.0
1.0
1.0
1.0


In [18]:
#联合评估
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

lr=LogisticRegression()
tree=DecisionTreeRegressor()
by = GaussianNB() 
svc=SVC(kernel='linear')
lr_pipe=Pipeline([('norm',norm),('clf',lr)])
lr_pipe.fit(x,y)
tree_pipe=Pipeline([('norm',norm),('clf',tree)])
tree_pipe.fit(x,y)
by_pipe=Pipeline([('norm',norm),('clf',by)])
by_pipe.fit(x,y)
svc_pipe=Pipeline([('norm',norm),('clf',svc)])
svc_pipe.fit(x,y)
print(lr_pipe.score(x,y))
print(tree_pipe.score(x,y))
print(by_pipe.score(x,y))
print(svc_pipe.score(x,y))

k=5
lr_pipe_score=cross_val_score(lr_pipe,x,y,cv=kf)
tree_pipe_score=cross_val_score(tree_pipe,x,y,cv=kf)
by_pipe_score=cross_val_score(by_pipe,x,y,cv=kf)
svc_pipe_score=cross_val_score(svc_pipe,x,y,cv=kf)
print(lr_pipe_score.mean())
print(tree_pipe_score.mean())
print(by_pipe_score.mean())
print(svc_pipe_score.mean())



0.6
1.0
0.7
0.6
0.3
-1.0
0.3
0.3


In [20]:
# auc/accuracy
from sklearn import metrics
lr_pipe.fit(xx,yy)
yyy=lr_pipe.predict(xx)
lr_auc=metrics.roc_auc_score(yy,yyy)
lr_accuracy=metrics.accuracy_score(yy,yyy)
[lr_auc,lr_accuracy]

[1.0, 1.0]